In [7]:
# Access MinIO files
param_minio_endpoint = "scruffy.lab.uvalight.net:9000"
param_minio_user_prefix = "zhanqing2016@gmail.com"  # Your personal folder in the naa-vre-user-data bucket in MinIO
secret_minio_access_key = "sFmE1jsm5hjJBBGh5RBL"
secret_minio_secret_key = "pczCG6FRpXQEtad7lAvXv00iCYFd5Dpa1g8GOWzR"

In [ ]:
# c1_TempSED_data
# ---
# NaaVRE:
#  cell:
#   inputs: []
#   outputs: 
#    - run_indices_sub: List
#   secrets:
#    - secret_s3_access_key: String
#    - secret_s3_secret_key: String
#   dependencies: 
#    - name: aws.s3
# ...


#   inputs:
#    - acolite_processing: List
# =========================
# Configure MinIO access
# =========================
param_minio_endpoint = "scruffy.lab.uvalight.net:9000"
param_minio_user_prefix = "zhanqing2016@gmail.com"  # Your personal folder in the naa-vre-user-data bucket in MinIO
secret_minio_access_key = ""
secret_minio_secret_key = ""


Sys.setenv("AWS_S3_ENDPOINT" = param_minio_endpoint,
           "AWS_DEFAULT_REGION" = "nl-uvalight",
           "AWS_ACCESS_KEY_ID" = secret_minio_access_key,
           "AWS_SECRET_ACCESS_KEY" = secret_minio_secret_key)

# =========================
# Load required libraries
# =========================
library(aws.s3)
library(devtools)
library(TempSED)
library(ReacTran)
library(pracma)
library(parallel)

# List existing buckets: get a list of all available buckets
#bucketlist()

# List files in bucket: get a list of files in a given bucket. For bucket `naa-vre-user-data`, only list files in your personal folder
#get_bucket_df(bucket="naa-vre-user-data", prefix=paste0(param_minio_user_prefix, "/"))


# =========================
# Helper function to download multiple files
# =========================
 download_rda_files <- function(bucket, folder, files, local_path=".") {
  if (!dir.exists(local_path)) dir.create(local_path, recursive = TRUE)
  for (file in files) {
    object_key <- paste0(folder, "/", file)
    local_file <- file.path(local_path, file)
    cat("Downloading:", object_key, "\n")
    save_object(bucket=bucket, object=object_key, file=local_file)
    cat("✓", file, "downloaded\n")
  }
}

# =========================
# Files to download from personal TempSED folder
# =========================
# Define your MinIO endpoint and bucket name
#bucket_name <- "naa-vre-user-data"  # Replace with your bucket name
minio_folder <- paste0(param_minio_user_prefix, "/TempSED/Ricky_data")  # Replace with your folder in the bucket
minio_dir_data <- "/home/jovyan/Cloud Storage/naa-vre-user-data/TempSED/Ricky_data" # To retrieve files directly from MinIO
input_folder <- "/tmp/input_dir"  # Replace with the local folder path
code_folder <- "/tmp/code"

# Ensure the local folder exists
if (!dir.exists(input_folder)) {
  dir.create(input_folder, recursive = TRUE)
}

if (!dir.exists(code_folder)) {
  dir.create(code_folder, recursive = TRUE)
}

# List of required .rda files (GB size)
example_data_files <- c(
  "out_WD_0.002_2020.rda",
  "out_WT_0.002_2020.rda"
)

# Download all required files from MinIO (big)
print("Downloading files from MinIO...")
download_rda_files(bucket=bucket_name, folder=minio_folder, files=example_data_files, local_path=input_folder)
print("All files downloaded successfully!")

# List all data
list.files(input_folder, full.names = TRUE, pattern = "\\.rda$")

file.copy(from = "/home/jovyan/Virtual Labs/Waddenzee proto DT/Heatwave_bivalve/Code/run_ensemble_2020.R", 
          to = paste0(code_folder,"/run_ensemble_2020.R"), overwrite = FALSE)
list.files(code_folder)

source(paste0(code_folder, "/run_ensemble_2020.R"))

run_indices_sub = list(run_indices2020[1:3114]) # change to list for workflow parallelization
class(run_indices_sub)

[1] "Loading data into R..."


: 

: 

: 

In [ ]:
# c2_TempSED_ensemble
# ---
# NaaVRE:
#  cell:
#   inputs: 
#    - run_indices_sub: List
#   outputs: [] 
#   secrets:
#    - secret_s3_access_key: String
#    - secret_s3_secret_key: String
#   dependencies: 
#    - name: aws.s3
#    - name: devtools
#    - name: TempSED
#    - name: ReacTran
#    - name: pracma
#    - name: parallel
# ...

library(aws.s3)
library(devtools)
library(TempSED)
library(ReacTran)
library(pracma)
library(parallel)

minio_dir_data <- "/home/jovyan/Cloud Storage/naa-vre-user-data/TempSED/Ricky_data" # To retrieve files directly from MinIO
input_folder <- "/tmp/input_dir"  # Replace with the local folder path
code_folder <- "/tmp/code"

source(paste0(code_folder, "/run_ensemble_2020.R"))

# =========================
# Load all .rda files automatically
# =========================
# Load all data

print("Loading data into R...")

example_data_files <- c(
  "obs2020.rda",
  "sed_pars2020.rda",
  "run_indices2020.rda",
  "Temp.ini_2020_fixedDeepT.rda"
)

for (file in example_data_files) {
  load(paste0(minio_dir_data, "/", file))
}

Big_data_files <- c(
  "out_WD_0.002_2020.rda",
  "out_WT_0.002_2020.rda"
)

for (file in Big_data_files) {
  load(paste0(input_folder, "/", file))
}

cat("All data loaded successfully!\n")

# =========================
# Extract columns from obs2020
# =========================
fWind.wad2020      <- obs2020[,c("Second", "windSpeed")]
fRad.wad2020       <- obs2020[,c("Second", "radiation")]
fTair.wad2020      <- obs2020[,c("Second", "airTemperature")]
fPair.wad2020      <- obs2020[,c("Second", "airPressure")]
fHumidity.wad2020  <- obs2020[,c("Second", "airHumidity")]
fCloud.wad2020     <- obs2020[,c("Second", "cloudCover")]

cat("obs2020 preview:\n")
print(head(obs2020))

# === Universal Parameters ===
z_max <- 10
dz_1     <- 1e-4
Grid = setup.grid.1D(N = 100, dx.1 = dz_1, L = z_max)

parms <- list(
  em_air = 0.8,
  em_sediment = 0.95,
  stanton = 0.001,
  dalton = 0.0014,
  density_water = 1024,
  density_solid = 2500,
  cp_water = 3994,
  tc_water = 0.6,
  tc_solid = 7,
  albedo_water = 0.05,
  kd_water = 1,
  kd_sediment = 1000
)

# === Model Run ===
output_dir <- "/tmp/output_dir/multicore/output_list_OS_2020_fixedDeepT_2yr"
#dir.create("../data/output_list_OS_2020_fixedDeepT_2yr", showWarnings = FALSE)

if(!dir.exists(output_dir)) dir.create(output_dir, showWarnings = FALSE, recursive = TRUE)


seconds_per_year <- 365 * 24 * 3600
times_1yr <- seq(from = 3600, by = 3600, length.out = 8760)
forcing_times_1yr <- seq(from = 3600, by = 600, length.out = 52560)

delta_U_threshold <- 0.001

# cores <- as.numeric(Sys.getenv("SLURM_CPUS_PER_TASK"))
cores <-3
out <- mclapply(seq_along(run_indices_sub), multicore_ensemble, mc.cores = cores)

cat("Number of ensemble members computed:\n")
length(out)

cat("Preview of first 3 outputs:\n")
print(out[1:3])


In [8]:
library(aws.s3)
library(devtools)
library(TempSED)
library(ReacTran)
library(pracma)
library(parallel)

?mclapply

Loading required package: usethis

Loading required package: deSolve

Loading required package: rootSolve

Loading required package: plot3D

Warning message:
“no DISPLAY variable so Tk is not available”
Loading required package: shape


Attaching package: ‘pracma’


The following objects are masked from ‘package:rootSolve’:

    gradient, hessian


The following object is masked from ‘package:deSolve’:

    rk4




mclapply               package:parallel                R Documentation

_P_a_r_a_l_l_e_l _V_e_r_s_i_o_n_s _o_f '_l_a_p_p_l_y' _a_n_d '_m_a_p_p_l_y' _u_s_i_n_g _F_o_r_k_i_n_g

_D_e_s_c_r_i_p_t_i_o_n:

     ‘mclapply’ is a parallelized version of ‘lapply’, it returns a
     list of the same length as ‘X’, each element of which is the
     result of applying ‘FUN’ to the corresponding element of ‘X’.

     It relies on forking and hence is not available on Windows unless
     ‘mc.cores = 1’.

     ‘mcmapply’ is a parallelized version of ‘mapply’, and ‘mcMap’
     corresponds to ‘Map’.

_U_s_a_g_e:

     mclapply(X, FUN, ...,
              mc.preschedule = TRUE, mc.set.seed = TRUE,
              mc.silent = FALSE, mc.cores = getOption("mc.cores", 2L),
              mc.cleanup = TRUE, mc.allow.recursive = TRUE, affinity.list = NULL)
     
     mcmapply(FUN, ...,
              MoreArgs = NULL, SIMPLIFY = TRUE, USE.NAMES = TRUE,
   